In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib

from dipy.core.gradients import gradient_table
from dipy.data import get_sphere, HemiSphere
from dipy.io import read_bvals_bvecs
from dipy.io.streamline import save_trk
from dipy.direction import (peaks_from_model, ProbabilisticDirectionGetter)
from dipy.reconst.gqi import GeneralizedQSamplingModel
from dipy.reconst.peaks import reshape_peaks_for_visualization
from dipy.tracking import utils
from dipy.tracking.local import (ThresholdTissueClassifier, LocalTracking,ActTissueClassifier)
from dipy.tracking.streamline import Streamlines
from dipy.tracking.utils import random_seeds_from_mask
from dipy.viz import window, actor, fvtk
from dipy.viz.colormap import line_colors
from dipy.io.image import save_nifti
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,auto_response)

Loading data

In [2]:
#dwi
fimg = ("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/10000/from_mgh/10k.nii.gz")
img = nib.load(fimg)
data = img.get_data()
affine = img.affine
#bval bvec 
fbval=("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/10000/from_mgh/bvals")
fbvec = ("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/10000/from_mgh/bvecs")
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab = gradient_table(bvals,bvecs)
print("Data 1k Shape: " + str(data.shape))

#importing brain mask
mask=("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/mask.nii.gz")
img = nib.load(mask)
brain_mask = img.get_data()


KeyboardInterrupt: 

Loading Sphere for discrete sampling directions

In [11]:
sphere = get_sphere('symmetric642')
sphere.vertices = sphere.vertices.astype('float')
sphere.edges = sphere.edges.astype('uint16')
sphere.faces = sphere.faces.astype('uint16')
sphere.phi = sphere.phi.astype('float')

Fitting GQI

In [12]:
#Instantiate the model and apply it to the data.
gqmodel = GeneralizedQSamplingModel(gtab, sampling_length=0.5)

gqi = peaks_from_model(model=gqmodel,
                           data=data,
                           sphere=sphere,
                           relative_peak_threshold=.5,        
                           min_separation_angle=25,
                           mask=brain_mask, return_odf=True,
                           normalize_peaks=True)

In [ ]:
Visualize Peaks

In [36]:
#Enables/disables interactive visualization
interactive = True
ren = window.Renderer()
window.clear(ren)
gq_peaks = actor.peak_slicer(gqi.peak_dirs, gqi.peak_values, colors=None)
ren.set_camera(position=(0, 10, 0))
ren.add(gq_peaks)

if interactive:
    window.show(ren)

In [59]:
#Compute GFA
GFA = gqi.gfa
save_nifti('/space/hemera/1/users/cmaffei/habenoula_mgh_1010/gfa_map.nii.gz', GFA, affine)

In [ ]:
Fitting CSD

In [3]:
response, ratio = auto_response(
        gtab, data, roi_radius=10, fa_thr=0.7)
csd_model = ConstrainedSphericalDeconvModel(gtab, response)

sphere = get_sphere('symmetric724')

csd = peaks_from_model(
        model=csd_model, data=data, sphere=sphere, mask=brain_mask,
        relative_peak_threshold=.5, min_separation_angle=25,
        parallel=True, return_sh=True)

##Tractography

Set seeds and Tissue classifier

In [4]:
#Load habenoula and VTA masks
mask=("/cluster/scratch/friday/ay/mgh_1010/avg_VTA_L_R_thr_60_bin_mul_alpha_04_all_grp_mask.nii.gz")
img = nib.load(mask)
VTA = img.get_data()
print("VTA Shape: " + str(VTA.shape))

mask=("/cluster/scratch/friday/ay/mgh_1010/Hb_mask_peak_voxel.nii.gz")
img = nib.load(mask)
HB = img.get_data()
print("HB Shape: " + str(HB.shape))

VTA Shape: (140, 140, 96)
HB Shape: (140, 140, 96)


In [5]:
#local tracker assumes that the data is sampled on a regular grid. seeds(same space as the affine!). 
seed_HB = random_seeds_from_mask(HB, seeds_count=1000)
seed_VTA = random_seeds_from_mask(VTA, seeds_count=1000)

#Define a tissue classifier. Streamlines will stop propagating for GFA < .25
# tissue_classifier = ThresholdTissueClassifier(GFA, .25)

Set ACT classifier

In [6]:
#Import PVE maps
img_pve_gm = nib.load('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/structural/T12diff_pve_1_res_con.nii.gz')
img_pve_csf = nib.load('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/structural/T12diff_pve_0_res_con.nii.gz')
img_pve_wm = nib.load('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/structural/T12diff_pve_2_res_con.nii.gz')

# The background of the anatomical image should be added to the include_map
# to keep streamlines exiting the brain (e.g. through the brain stem). The ACT tissue classifier uses a trilinear 
# interpolation at the tracking position.
background = np.ones(img_pve_gm.shape)
background[(img_pve_gm.get_data() +
            img_pve_wm.get_data() +
            img_pve_csf.get_data()) > 0] = 0
                       
include_map = img_pve_gm.get_data()
include_map[background > 0] = 1
exclude_map = img_pve_csf.get_data()

act_classifier = ActTissueClassifier(include_map, exclude_map)

Deterministic Tractography CSD

In [7]:
# streamline_generator = LocalTracking(
#         csd, tissue_classifier, seed_VTA, affine=np.eye(4),
#         step_size=0.75)
# streamlines = Streamlines(streamline_generator)
# #Save the trk file
# save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/10k/vta_1000seeds_det_csd_tc.trk",
#          streamlines,
#          img.affine,
#          shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

# streamline_generator = LocalTracking(
#         csd, tissue_classifier, seed_HB, affine=np.eye(4),
#         step_size=0.75)
# streamlines = Streamlines(streamline_generator)
# #Save the trk file
# save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/10k/hb_1000seeds_det_csd_tc.trk",
#          streamlines,
#          img.affine,
#          shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

streamline_generator = LocalTracking(
        csd, act_classifier, seed_VTA, affine=np.eye(4),
        step_size=0.75)
streamlines = Streamlines(streamline_generator)
#Save the trk file
save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/10k/vta_1000seeds_det_csd_act.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

streamline_generator = LocalTracking(
        csd, act_classifier, seed_HB, affine=np.eye(4),
        step_size=0.75)
streamlines = Streamlines(streamline_generator)
#Save the trk file
save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/10k/hb_1000seeds_det_csd_act.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])



Probabilistic Tractography CSD

In [9]:
prob_dg = ProbabilisticDirectionGetter.from_shcoeff(csd.shm_coeff, max_angle=30, sphere=sphere)

# streamline_generator = LocalTracking(
#         prob_dg, tissue_classifier, seed_VTA, affine=np.eye(4),
#         step_size=0.75)
# streamlines = Streamlines(streamline_generator)
# #Save the trk file
# save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/10k/vta_1000seeds_prob_csd_tc.trk",
#          streamlines,
#          img.affine,
#          shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

# streamline_generator = LocalTracking(
#         prob_dg, tissue_classifier, seed_HB, affine=np.eye(4),
#         step_size=0.75)
# streamlines = Streamlines(streamline_generator)
# #Save the trk file
# save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/10k/hb_1000seeds_prob_csd_tc.trk",
#          streamlines,
#          img.affine,
#          shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

streamline_generator = LocalTracking(
        prob_dg, act_classifier, seed_VTA, affine=np.eye(4),
        step_size=0.75)
streamlines = Streamlines(streamline_generator)
#Save the trk file
save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/10k/vta_1000seeds_prob_csd_act.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

streamline_generator = LocalTracking(
        prob_dg, act_classifier, seed_HB, affine=np.eye(4),
        step_size=0.75)
streamlines = Streamlines(streamline_generator)
#Save the trk file
save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/10k/hb_1000seeds_prob_csd_act.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

Deterministic Tractography GQI

In [13]:
# streamline_generator = LocalTracking(
#         gqi, tissue_classifier, seed_VTA, affine=np.eye(4),
#         step_size=0.75)
# streamlines = Streamlines(streamline_generator)
# #Save the trk file
# save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/10k/vta_1000seeds_det_gqi_tc.trk",
#          streamlines,
#          img.affine,
#          shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

# streamline_generator = LocalTracking(
#         gqi, tissue_classifier, seed_HB, affine=np.eye(4),
#         step_size=0.75)
# streamlines = Streamlines(streamline_generator)
# #Save the trk file
# save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/10k/hb_1000seeds_det_gqi_tc.trk",
#          streamlines,
#          img.affine,
#          shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

streamline_generator = LocalTracking(
        gqi, act_classifier, seed_VTA, affine=np.eye(4),
        step_size=0.75)
streamlines = Streamlines(streamline_generator)
#Save the trk file
save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/10k/vta_1000seeds_det_gqi_act.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

streamline_generator = LocalTracking(
        gqi, act_classifier, seed_HB, affine=np.eye(4),
        step_size=0.75)
streamlines = Streamlines(streamline_generator)
#Save the trk file
save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/10k/hb_1000seeds_det_gqi_act.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])


Probabilistic Tractography GQI 

In [49]:
pmf = gqi.odf.clip(min=0)
prob_dg = ProbabilisticDirectionGetter.from_pmf(pmf, max_angle=30, sphere=sphere)

streamline_generator = LocalTracking(
        prob_dg, tissue_classifier, seed_VTA, affine=np.eye(4),
        step_size=0.75)
streamlines = Streamlines(streamline_generator)
#Save the trk file
save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/10k/vta_1000seeds_prob_gqi_tc.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

streamline_generator = LocalTracking(
        prob_dg, tissue_classifier, seed_HB, affine=np.eye(4),
        step_size=0.75)
streamlines = Streamlines(streamline_generator)
#Save the trk file
save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/10k/hb_1000seeds_prob_gqi_tc.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

streamline_generator = LocalTracking(
        prob_dg, act_classifier, seed_VTA, affine=np.eye(4),
        step_size=0.75)
streamlines = Streamlines(streamline_generator)
#Save the trk file
save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/10k/vta_1000seeds_prob_gqi_act.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

streamline_generator = LocalTracking(
        prob_dg, act_classifier, seed_HB, affine=np.eye(4),
        step_size=0.75)
streamlines = Streamlines(streamline_generator)
#Save the trk file
save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/10k/hb_1000seeds_prob_gqi_act.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])


BootStrap GQI

In [67]:
from dipy.direction import BootDirectionGetter
from dipy.data import small_sphere

boot_dg_gq = BootDirectionGetter.from_data(data, gqmodel, max_angle=30.,
                                           sphere=sphere)

boot_streamline_generator = LocalTracking(boot_dg_gq, tissue_classifier, seed_HB,
                                          affine=np.eye(4), step_size=.75)

streamlines = Streamlines(boot_streamline_generator)

#Save the trk file
save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/1k/hb_1000seeds_gqi_bootstrap_tc.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

boot_streamline_generator = LocalTracking(boot_dg_gq, tissue_classifier, seed_VTA,
                                          affine=np.eye(4), step_size=.75)

streamlines = Streamlines(boot_streamline_generator)

#Save the trk file
save_trk("/space/hemera/1/users/cmaffei/habenoula_mgh_1010/1k/vta_1000seeds_gqi_bootstrap_tc.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])
